In [ ]:
#导入 selenium 第三方库
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import os
import glob
import re
from pathlib import Path 

DOWNLOAD_DIR = os.path.abspath("Information Sciences")  # 相对路径
chrome_options = Options()
prefs = {
    # 设置默认下载路径
    "download.default_directory": DOWNLOAD_DIR,
    # 禁用下载前的文件保存确认弹窗
    "download.prompt_for_download": False,
    # 关闭PDF文件的内置预览（直接下载而非在浏览器打开）
    "plugins.always_open_pdf_externally": True,
    # 禁用下载通知栏提示
    "download.directory_upgrade": True
}
chrome_options.add_experimental_option("prefs", prefs)

In [ ]:
#指定驱动路径
service = Service(r"./chromedriver.exe")        
driver = webdriver.Chrome(service=service,  options=chrome_options)

In [ ]:
# -------------------------- 工具函数：等待下载完成 + 重命名 --------------------------
def wait_for_download_complete(download_dir, timeout=60):
    """
    等待本次下载完成，返回最新下载的文件路径（仅识别本次新增的文件）
    :param download_dir: 下载目录（绝对路径，建议用r前缀）
    :param timeout: 超时时间（秒）
    :return: 本次下载的文件路径
    """
    # 步骤1：记录「下载开始前」的基准文件列表（区分旧文件）
    download_dir = Path(download_dir)  # 用Path简化路径操作
    # 基准列表：下载前已存在的非临时文件（排除所有下载临时后缀）
    temp_suffixes = (".crdownload", ".part", ".tmp")  # 覆盖主流浏览器临时文件
    baseline_files = set()
    for file in download_dir.iterdir():
        if file.is_file() and not file.name.lower().endswith(temp_suffixes):
            baseline_files.add(file.name)

    start_time = time.time()
    latest_new_file = None  # 存储本次下载的新文件
    
    while time.time() - start_time < timeout:
        # 步骤2：遍历目录，筛选「本次新增的文件」（不在基准列表+非临时文件）
        current_files = []
        for file in download_dir.iterdir():
            # 排除临时文件、文件夹、旧文件
            if (file.is_file() 
                and not file.name.lower().endswith(temp_suffixes)
                and file.name not in baseline_files):
                current_files.append(file)
        
        if current_files:
            # 取本次新增文件中「最新修改」的（确保是刚下载完成的）
            latest_new_file = max(current_files, key=lambda x: x.stat().st_mtime)
            # 额外保险：等待文件大小稳定（避免文件还在写入）
            prev_size = -1
            stable_count = 0
            while stable_count < 2:  # 连续2次检查大小不变
                current_size = latest_new_file.stat().st_size
                if current_size == prev_size:
                    stable_count += 1
                else:
                    prev_size = current_size
                    stable_count = 0
                time.sleep(0.5)
            return str(latest_new_file)  # 返回绝对路径
        
        # 步骤3：如果还没新文件，等待1秒再检查（不着急返回）
        time.sleep(1)
    
    # 当前页面刷新, 再尝试
    driver.refresh()
    return None  # 超时未找到新文件


In [ ]:
def rename_downloaded_file(old_file_path, new_file_path):
    """
    重命名下载的文件（处理同名文件覆盖）
    :param old_file_path: 原文件路径
    :param new_file_path: 新文件路径
    """
    # 如果新文件已存在，先删除（可选：也可以加后缀避免覆盖）
    if os.path.exists(new_file_path):
        os.remove(new_file_path)
    # 重命名
    os.rename(old_file_path, new_file_path)
    print(f"文件已重命名：{new_file_path}")
    


In [ ]:
def is_file_downloaded(download_dir, target_filename, timeout=60):
    """
    判断文件是否下载完成
    """
    start_time = time.time()
    temp_suffixes = [".crdownload", ".part", ".tmp"]  # 浏览器临时文件后缀
    
    while time.time() - start_time < timeout:
        # 遍历下载目录下的所有文件
        for filename in os.listdir(download_dir):
            # 情况1：找到最终文件（无临时后缀）
            if filename == target_filename:
                time.sleep(1)  # 保险：等待文件写入完成
                return True
            # 情况2：找到临时文件（还在下载中）
            elif filename.startswith(target_filename) and any(filename.endswith(suf) for suf in temp_suffixes):
                time.sleep(0.5)  # 下载中，继续等待
                break
        else:
            # 既没找到最终文件，也没找到临时文件 → 可能还没开始下载
            time.sleep(0.5)
    # 超时
    return False

In [ ]:
def wait_for_download_complete(download_dir, timeout=60):
    """
    等待本次下载完成，返回最新下载的文件路径（仅识别本次新增的文件）
    :param download_dir: 下载目录（绝对路径，建议用r前缀）
    :param timeout: 超时时间（秒）
    :return: 本次下载的文件路径
    """
    # 步骤1：记录「下载开始前」的基准文件列表（区分旧文件）
    download_dir = Path(download_dir)  # 用Path简化路径操作
    # 基准列表：下载前已存在的非临时文件（排除所有下载临时后缀）
    temp_suffixes = (".crdownload", ".part", ".tmp")  # 覆盖主流浏览器临时文件
    baseline_files = set()
    for file in download_dir.iterdir():
        if file.is_file() and not file.name.lower().endswith(temp_suffixes):
            baseline_files.add(file.name)

    start_time = time.time()
    latest_new_file = None  # 存储本次下载的新文件
    
    while time.time() - start_time < timeout:
        # 步骤2：遍历目录，筛选「本次新增的文件」（不在基准列表+非临时文件）
        current_files = []
        for file in download_dir.iterdir():
            # 排除临时文件、文件夹、旧文件
            if (file.is_file() 
                and not file.name.lower().endswith(temp_suffixes)
                and file.name not in baseline_files):
                current_files.append(file)
        
        if current_files:
            # 取本次新增文件中「最新修改」的（确保是刚下载完成的）
            latest_new_file = max(current_files, key=lambda x: x.stat().st_mtime)
            # 额外保险：等待文件大小稳定（避免文件还在写入）
            prev_size = -1
            stable_count = 0
            while stable_count < 2:  # 连续2次检查大小不变
                current_size = latest_new_file.stat().st_size
                if current_size == prev_size:
                    stable_count += 1
                else:
                    prev_size = current_size
                    stable_count = 0
                time.sleep(0.5)
            return str(latest_new_file)  # 返回绝对路径
        
        # 步骤3：如果还没新文件，等待1秒再检查（不着急返回）
        time.sleep(1)
    
    # 超时抛出异常
    raise TimeoutError(
        f"下载超时！{timeout}秒内未找到本次下载的新文件\n"
        f"下载目录已有旧文件：{list(baseline_files)[:5]}（最多显示5个）"
    )

In [ ]:
# -------------------------- 主逻辑 --------------------------
# 创建下载目录（如果不存在）
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)




# 访问目标页面
driver.get("https://www.sciengine.com/SCIS/issue/56/12")


while True:
    #选择全部
    time.sleep(3)  # 等待页面加载
    select_all_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="journal-list"]/div[1]/div[1]/div[3]/div[1]'))
    )
    select_all_button.click()

    # 点击下载按钮
    download_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="journal-list"]/div[1]/div[1]/div[3]/div[4]'))  # 关键：外层加()组成元组
    )
    download_button.click()

    # 等待下载完成并获取新文件路径
    downloaded_file_path = wait_for_download_complete(DOWNLOAD_DIR, timeout=360)
    print(f"下载完成，文件路径：{downloaded_file_path}")

    #点击上一页
    next_page_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="journal-list"]/div[1]/div[1]/div[2]/div[2]/div[2]/i[1]'))  # 关键：外层加()组成元组
    )
    next_page_button.click()